# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [90]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [91]:
weather_csv_df = "weather.csv"
weather_df = pd.read_csv(weather_csv_df)
weather_df = weather_df.drop(weather_df.columns[[0]], axis=1) 
weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,thompson,55.74,-97.86,12.00,76.0,75.0,2.10,CA,1.598727e+09
1,banda aceh,5.56,95.32,26.61,78.0,91.0,1.67,ID,1.598727e+09
2,carutapera,-1.20,-46.02,29.66,63.0,28.0,5.44,BR,1.598727e+09
3,khatanga,71.97,102.50,9.53,74.0,62.0,3.69,RU,1.598727e+09
4,carnarvon,-24.87,113.63,16.00,67.0,0.0,5.70,AU,1.598727e+09
...,...,...,...,...,...,...,...,...,...
616,mizdah,31.45,12.98,29.40,18.0,0.0,4.96,LY,1.598727e+09
617,curup,-3.47,102.53,20.31,90.0,96.0,1.62,ID,1.598727e+09
618,eyl,7.98,49.82,25.65,75.0,0.0,10.02,SO,1.598727e+09
619,belyy yar,53.60,91.39,12.00,93.0,0.0,1.00,RU,1.598727e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [92]:
gmaps.configure(api_key=g_key)

In [98]:
locations = weather_df[["Lat", "Lng"]].astype(float)
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

In [99]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
vacation_df = weather_df.loc[(weather_df['Max Temp'] > 20) & (weather_df['Max Temp'] < 40) & (weather_df['Wind Speed'] < 5) & (weather_df['Cloudiness'] == 0)]
vacation_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,shingu,33.73,135.98,24.43,88.0,0.0,1.67,JP,1.598727e+09
76,gat,31.61,34.76,28.89,77.0,0.0,2.17,IL,1.598727e+09
81,morondava,-20.28,44.28,23.38,87.0,0.0,3.05,MG,1.598727e+09
84,abu samrah,35.30,37.18,27.68,17.0,0.0,1.32,SY,1.598727e+09
99,mahajanga,-15.72,46.32,23.00,78.0,0.0,1.00,MG,1.598727e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [61]:
hotel_df = vacation_df
hotel_df['Hotel Name']= ""
hotel_df['Hotel Lat'] = ""
hotel_df['Hotel Lng'] = ""
hotel_df.head()

C:\Users\krist\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\krist\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\krist\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
9,shingu,33.73,135.98,24.43,88.0,0.0,1.67,JP,1.598727e+09,,,
76,gat,31.61,34.76,28.89,77.0,0.0,2.17,IL,1.598727e+09,,,
81,morondava,-20.28,44.28,23.38,87.0,0.0,3.05,MG,1.598727e+09,,,
84,abu samrah,35.30,37.18,27.68,17.0,0.0,1.32,SY,1.598727e+09,,,
99,mahajanga,-15.72,46.32,23.00,78.0,0.0,1.00,MG,1.598727e+09,,,


In [81]:
for index,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
  
    target_type = "lodging"
    radius = 5000
    
    params = {
        "location": f"{lat},{lng}",
        "types": target_type,
        "radius": radius,
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params).json()
    results = response['results']
#     print(response['name'])  
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, "Hotel Lat"] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, "Hotel Lng"] = results[0]['geometry']['location']['lng']
    except (IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "NaN"
        hotel_df.loc[index, "Hotel Lat"] = "NaN"
        hotel_df.loc[index, "Hotel Lng"] = "NaN"
    

    
hotel_df.head()
    



C:\Users\krist\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
9,shingu,33.73,135.98,24.43,88.0,0.0,1.67,JP,1.598727e+09,Shingu UI Hotel,33.7227,135.992
76,gat,31.61,34.76,28.89,77.0,0.0,2.17,IL,1.598727e+09,רגע בנחת,31.6483,34.7374
81,morondava,-20.28,44.28,23.38,87.0,0.0,3.05,MG,1.598727e+09,La Case Bambou,-20.2979,44.2694
84,abu samrah,35.30,37.18,27.68,17.0,0.0,1.32,SY,1.598727e+09,NaN,NaN,NaN
99,mahajanga,-15.72,46.32,23.00,78.0,0.0,1.00,MG,1.598727e+09,Restaurant Greedy Coco Lodge Majunga,-15.7224,46.3074


In [85]:
# Get rid of blank rows so it doesn't mess up next section
hotel_df = hotel_df[hotel_df['Hotel Name'] != "NaN"]
hotel_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
9,shingu,33.73,135.98,24.43,88.0,0.0,1.67,JP,1.598727e+09,Shingu UI Hotel,33.7227,135.992
76,gat,31.61,34.76,28.89,77.0,0.0,2.17,IL,1.598727e+09,רגע בנחת,31.6483,34.7374
81,morondava,-20.28,44.28,23.38,87.0,0.0,3.05,MG,1.598727e+09,La Case Bambou,-20.2979,44.2694
99,mahajanga,-15.72,46.32,23.00,78.0,0.0,1.00,MG,1.598727e+09,Restaurant Greedy Coco Lodge Majunga,-15.7224,46.3074
123,hays,30.05,-98.03,37.22,41.0,0.0,1.79,US,1.598727e+09,Sinya on Lone Man Creek,30.0282,-98.0444


In [95]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_hotel = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [103]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations_hotel, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Nearest lodgin option: {hotel_df.loc[index, 'Hotel Name']}" for index,row in hotel_df.iterrows()]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))